# Party Voting Loyalty Analysis

Analyze party cohesion and individual voting behavior in Bundestag votes.

In [ ]:
# Import libraries
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import yaml

# Database connection
import sys
sys.path.insert(0, str(Path.cwd().parent / 'src'))
from xminer.io.db import engine
from sqlalchemy import text

print('✅ Libraries imported successfully')

In [ ]:
# Configuration
PARAMS_FILE = Path("../src/xminer/config/parameters.yml")

with PARAMS_FILE.open("r", encoding="utf-8") as f:
    params = yaml.safe_load(f) or {}

YEAR = int(params.get("year", 2025))
MONTH = int(params.get("month", 12))
YM = f"{YEAR:04d}{MONTH:02d}"

# Graphics directory
GRAPHICS_BASE_DIR = Path(params.get("graphics_base_dir", "../outputs"))
GRAPHICS_DIR = GRAPHICS_BASE_DIR / YM / "graphics" / "voting_loyalty"
GRAPHICS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Output: {GRAPHICS_DIR}")

In [ ]:
# Party colors (standard German party colors)
PARTY_COLORS = {
    "CDU/CSU": "#000000",
    "SPD": "#E3000F",
    "BÜ90/GR": "#1AA64A",
    "FDP": "#FFED00",
    "AfD": "#009EE0",
    "Die Linke": "#BE3075",
    "BSW": "#009688",
}

def get_party_color(party: str) -> str:
    return PARTY_COLORS.get(party, "#888888")

## 1. Load Data from Analysis Views

In [ ]:
# Get party cohesion data
query_cohesion = """
SELECT *
FROM party_voting_cohesion
ORDER BY avg_cohesion_percent DESC
"""

with engine.connect() as conn:
    df_cohesion = pd.read_sql(text(query_cohesion), conn)

print(f"Loaded {len(df_cohesion)} parties")
df_cohesion

In [ ]:
# Get most rebellious politicians
query_rebellious = """
SELECT *
FROM party_rebellious_politicians
WHERE dissent_rank <= 5
ORDER BY party, dissent_rank
"""

with engine.connect() as conn:
    df_rebellious = pd.read_sql(text(query_rebellious), conn)

# Create full name
df_rebellious = df_rebellious.assign(
    full_name=df_rebellious['vorname'] + ' ' + df_rebellious['name']
)

print(f"Loaded {len(df_rebellious)} rebellious politicians")
df_rebellious.head(10)

In [ ]:
# Get overall dissent rates
query_dissent = """
SELECT 
    party,
    COUNT(*) as politician_count,
    AVG(dissent_rate) as avg_dissent_rate,
    MAX(dissent_rate) as max_dissent_rate,
    MIN(dissent_rate) as min_dissent_rate
FROM politician_voting_loyalty
WHERE total_votes_participated >= 10
GROUP BY party
ORDER BY avg_dissent_rate DESC
"""

with engine.connect() as conn:
    df_dissent = pd.read_sql(text(query_dissent), conn)

print(f"Loaded dissent rates for {len(df_dissent)} parties")
df_dissent

In [ ]:
# Get party cohesion over time (timeline data)
query_timeline = """
SELECT 
    vote_date,
    party,
    cohesion_percent,
    vote_title,
    party_yes_votes,
    party_no_votes,
    party_abstain_votes
FROM party_vote_positions
WHERE vote_date IS NOT NULL
ORDER BY vote_date, party
"""

with engine.connect() as conn:
    df_timeline = pd.read_sql(text(query_timeline), conn)

# Convert vote_date to datetime
df_timeline['vote_date'] = pd.to_datetime(df_timeline['vote_date'])

print(f"Loaded {len(df_timeline)} vote-party combinations")
print(f"Date range: {df_timeline['vote_date'].min()} to {df_timeline['vote_date'].max()}")
df_timeline.head(10)

In [ ]:
# Create bar chart of party cohesiondef create_party_cohesion_plot(df, language='de'):        if language == 'de':        title = "<b>Partei-Geschlossenheit bei Abstimmungen</b><br><sub>Durchschnittliche Übereinstimmung mit Parteimehrheit (29.01. - 19.12.2025)</sub>"        yaxis_title = "<b>Durchschnittliche Geschlossenheit (%)</b>"    else:        title = "<b>Party Voting Cohesion</b><br><sub>Average Agreement with Party Majority (29 Jan - 19 Dec 2025)</sub>"        yaxis_title = "<b>Average Cohesion (%)</b>"        # Sort by cohesion    df_plot = df.sort_values('avg_cohesion_percent', ascending=True)        colors = [get_party_color(p) for p in df_plot['party']]        fig = go.Figure()        fig.add_trace(go.Bar(        y=df_plot['party'],        x=df_plot['avg_cohesion_percent'],        orientation='h',        marker_color=colors,        text=[f"<b>{val:.1f}%</b>" for val in df_plot['avg_cohesion_percent']],        textposition='outside',        textfont=dict(color='white', size=16),        hovertemplate=(            "<b>%{y}</b><br>"            "Cohesion: %{x:.1f}%<br>"            "<extra></extra>"        )    ))        fig.update_layout(        title=dict(            text=title,            x=0.5,            xanchor='center',            font=dict(size=28, color='white')        ),        plot_bgcolor='#1a1a1a',        paper_bgcolor='#1a1a1a',        font=dict(color='white', size=14, family='Arial'),        height=1080,  # Instagram square        width=1080,        showlegend=False,        margin=dict(b=100, t=140, l=150, r=150),        xaxis=dict(            title=yaxis_title,            gridcolor='#333333',            title_font=dict(size=18),            tickfont=dict(size=14),            range=[95, 100.5]  # Zoom in since all are high        ),        yaxis=dict(            gridcolor='#333333',            tickfont=dict(size=16)        )    )        return fig# Create and savefig_de = create_party_cohesion_plot(df_cohesion, 'de')fig_en = create_party_cohesion_plot(df_cohesion, 'en')output_de = GRAPHICS_DIR / "party_cohesion_de.png"output_en = GRAPHICS_DIR / "party_cohesion_en.png"fig_de.write_image(output_de, width=1080, height=1080, scale=2)fig_en.write_image(output_en, width=1080, height=1080, scale=2)print(f"✅ Saved: {output_de}")print(f"✅ Saved: {output_en}")fig_de.show()

In [ ]:
# Create bar chart of dissent ratesdef create_dissent_rate_plot(df, language='de'):        if language == 'de':        title = "<b>Durchschnittliche Abweichungsrate nach Partei</b><br><sub>Wie oft stimmen Abgeordnete anders als ihre Partei? (29.01. - 19.12.2025)</sub>"        yaxis_title = "<b>Durchschn. Abweichungsrate (%)</b>"    else:        title = "<b>Average Dissent Rate by Party</b><br><sub>How often do MPs vote differently than their party? (29 Jan - 19 Dec 2025)</sub>"        yaxis_title = "<b>Avg. Dissent Rate (%)</b>"        # Sort by dissent rate descending    df_plot = df.sort_values('avg_dissent_rate', ascending=False)        colors = [get_party_color(p) for p in df_plot['party']]        fig = go.Figure()        fig.add_trace(go.Bar(        x=df_plot['party'],        y=df_plot['avg_dissent_rate'],        marker_color=colors,        text=[f"<b>{val:.1f}%</b>" for val in df_plot['avg_dissent_rate']],        textposition='outside',        textfont=dict(color='white', size=16),        hovertemplate=(            "<b>%{x}</b><br>"            "Avg Dissent: %{y:.1f}%<br>"            "Max Dissent: " + df_plot['max_dissent_rate'].astype(str) + "%<br>"            "<extra></extra>"        )    ))        fig.update_layout(        title=dict(            text=title,            x=0.5,            xanchor='center',            font=dict(size=28, color='white')        ),        plot_bgcolor='#1a1a1a',        paper_bgcolor='#1a1a1a',        font=dict(color='white', size=14, family='Arial'),        height=1080,        width=1080,        showlegend=False,        margin=dict(b=120, t=140, l=100, r=100),        xaxis=dict(            gridcolor='#333333',            tickfont=dict(size=15)        ),        yaxis=dict(            title=yaxis_title,            gridcolor='#333333',            title_font=dict(size=18),            tickfont=dict(size=14)        )    )        return fig# Create and savefig_de = create_dissent_rate_plot(df_dissent, 'de')fig_en = create_dissent_rate_plot(df_dissent, 'en')output_de = GRAPHICS_DIR / "dissent_rate_by_party_de.png"output_en = GRAPHICS_DIR / "dissent_rate_by_party_en.png"fig_de.write_image(output_de, width=1080, height=1080, scale=2)fig_en.write_image(output_en, width=1080, height=1080, scale=2)print(f"✅ Saved: {output_de}")print(f"✅ Saved: {output_en}")fig_de.show()

In [ ]:
# Create timeline plot showing party cohesion over time
def create_cohesion_timeline_plot(df, language='de'):
    
    if language == 'de':
        title = "<b>Partei-Geschlossenheit im Zeitverlauf 2025</b><br><sub>Wie geschlossen stimmten Parteien bei jeder Abstimmung?</sub>"
        xaxis_title = "<b>Datum</b>"
        yaxis_title = "<b>Geschlossenheit (%)</b>"
    else:
        title = "<b>Party Cohesion Timeline 2025</b><br><sub>How cohesively did parties vote in each vote?</sub>"
        xaxis_title = "<b>Date</b>"
        yaxis_title = "<b>Cohesion (%)</b>"
    
    # Filter to main parties
    main_parties = ['AfD', 'CDU/CSU', 'SPD', 'BÜ90/GR', 'Die Linke']
    df_plot = df[df['party'].isin(main_parties)].copy()
    
    fig = go.Figure()
    
    # Add a line for each party
    for party in main_parties:
        df_party = df_plot[df_plot['party'] == party].sort_values('vote_date')
        
        party_color = get_party_color(party)
        
        # Create hover text with vote details
        hover_texts = [
            f"<b>{party}</b><br>"
            f"Datum: {row['vote_date'].strftime('%d.%m.%Y')}<br>"
            f"Geschlossenheit: {row['cohesion_percent']:.1f}%<br>"
            f"Abstimmung: {row['vote_title'][:60]}...<br>"
            f"Ja: {row['party_yes_votes']}, Nein: {row['party_no_votes']}, Enth: {row['party_abstain_votes']}"
            for _, row in df_party.iterrows()
        ]
        
        fig.add_trace(go.Scatter(
            x=df_party['vote_date'],
            y=df_party['cohesion_percent'],
            mode='lines+markers',
            name=party,
            line=dict(color=party_color, width=3),
            marker=dict(size=8, color=party_color, line=dict(color='white', width=1)),
            customdata=hover_texts,
            hovertemplate='%{customdata}<extra></extra>'
        ))
    
    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            xanchor='center',
            font=dict(size=26, color='white')
        ),
        plot_bgcolor='#1a1a1a',
        paper_bgcolor='#1a1a1a',
        font=dict(color='white', size=14, family='Arial'),
        height=1350,  # Instagram portrait
        width=1080,
        margin=dict(b=120, t=140, l=100, r=100),
        xaxis=dict(
            title=xaxis_title,
            gridcolor='#333333',
            title_font=dict(size=18),
            tickfont=dict(size=13),
            tickangle=-45
        ),
        yaxis=dict(
            title=yaxis_title,
            gridcolor='#333333',
            title_font=dict(size=18),
            tickfont=dict(size=14),
            range=[85, 102]  # Zoom in since cohesion is high
        ),
        legend=dict(
            x=0.02,
            y=0.98,
            bgcolor='rgba(26, 26, 26, 0.8)',
            bordercolor='white',
            borderwidth=1,
            font=dict(size=14)
        ),
        hovermode='closest'
    )
    
    return fig

# Create and save
fig_de = create_cohesion_timeline_plot(df_timeline, 'de')
fig_en = create_cohesion_timeline_plot(df_timeline, 'en')

output_de = GRAPHICS_DIR / "cohesion_timeline_de.png"
output_en = GRAPHICS_DIR / "cohesion_timeline_en.png"

fig_de.write_image(output_de, width=1080, height=1350, scale=2)
fig_en.write_image(output_en, width=1080, height=1350, scale=2)

print(f"✅ Saved: {output_de}")
print(f"✅ Saved: {output_en}")

fig_de.show()

## 3.5 Visualization: Party Cohesion Timeline (2025)

In [ ]:
# Create horizontal bar chart of rebellious politiciansdef create_rebellious_politicians_plot(df, party_name, language='de'):        # Filter for specific party    df_party = df[df['party'] == party_name].head(10)        if len(df_party) == 0:        print(f"No data for {party_name}")        return None        if language == 'de':        title = f"<b>Abweichende Abgeordnete: {party_name}</b><br><sub>Top 10 nach Abweichungsrate (29.01. - 19.12.2025)</sub>"        xaxis_title = "<b>Abweichungsrate (%)</b>"    else:        title = f"<b>Most Rebellious: {party_name}</b><br><sub>Top 10 by Dissent Rate (29 Jan - 19 Dec 2025)</sub>"        xaxis_title = "<b>Dissent Rate (%)</b>"        # Sort by dissent rate ascending for horizontal bar    df_plot = df_party.sort_values('dissent_rate', ascending=True)        party_color = get_party_color(party_name)        # Create hover text    hover_texts = [        f"<b>{row['full_name']}</b><br>"        f"Abweichungen: {row['voted_against_party']}/{row['total_votes_participated']} Abstimmungen<br>"        f"Abweichungsrate: {row['dissent_rate']:.1f}%"        for _, row in df_plot.iterrows()    ]        fig = go.Figure()        fig.add_trace(go.Bar(        y=df_plot['full_name'],        x=df_plot['dissent_rate'],        orientation='h',        marker_color=party_color,        customdata=hover_texts,        hovertemplate="%{customdata}<extra></extra>"    ))        fig.update_layout(        title=dict(            text=title,            x=0.5,            xanchor='center',            font=dict(size=28, color='white')        ),        plot_bgcolor='#1a1a1a',        paper_bgcolor='#1a1a1a',        font=dict(color='white', size=14, family='Arial'),        height=1350,  # Instagram portrait        width=1080,        showlegend=False,        margin=dict(b=100, t=140, l=320, r=100),        xaxis=dict(            title=xaxis_title,            gridcolor='#333333',            title_font=dict(size=18),            tickfont=dict(size=14)        ),        yaxis=dict(            gridcolor='#333333',            tickfont=dict(size=13),            automargin=True        )    )        return fig# Create for each major partyparties_to_plot = ['AfD', 'CDU/CSU', 'SPD', 'BÜ90/GR', 'Die Linke']for party in parties_to_plot:    fig_de = create_rebellious_politicians_plot(df_rebellious, party, 'de')    fig_en = create_rebellious_politicians_plot(df_rebellious, party, 'en')        if fig_de:        safe_party = party.replace('/', '_').replace(' ', '_')        output_de = GRAPHICS_DIR / f"rebellious_{safe_party}_de.png"        output_en = GRAPHICS_DIR / f"rebellious_{safe_party}_en.png"                fig_de.write_image(output_de, width=1080, height=1350, scale=2)        fig_en.write_image(output_en, width=1080, height=1350, scale=2)                print(f"✅ Saved: {output_de}")        print(f"✅ Saved: {output_en}")

## 5. Summary Statistics

In [ ]:
print("="*80)
print("VOTING LOYALTY SUMMARY")
print("="*80)

print("\n1. Party Cohesion:")
print("-"*80)
for _, row in df_cohesion.iterrows():
    print(f"{row['party']:12} | Avg Cohesion: {row['avg_cohesion_percent']:>5.1f}% | "
          f"High Cohesion Rate: {row['high_cohesion_rate']:>5.1f}% | "
          f"Total Votes: {row['total_votes']:>2}")

print("\n2. Average Dissent by Party:")
print("-"*80)
for _, row in df_dissent.iterrows():
    print(f"{row['party']:12} | Avg Dissent: {row['avg_dissent_rate']:>5.1f}% | "
          f"Max Dissent: {row['max_dissent_rate']:>5.1f}% | "
          f"Politicians: {row['politician_count']:>3}")

print("\n3. Key Findings:")
print("-"*80)
print(f"Most cohesive party: {df_cohesion.iloc[0]['party']} ({df_cohesion.iloc[0]['avg_cohesion_percent']:.1f}%)")
print(f"Least cohesive party: {df_cohesion.iloc[-1]['party']} ({df_cohesion.iloc[-1]['avg_cohesion_percent']:.1f}%)")
print(f"Highest avg dissent: {df_dissent.iloc[0]['party']} ({df_dissent.iloc[0]['avg_dissent_rate']:.1f}%)")
print(f"Lowest avg dissent: {df_dissent.iloc[-1]['party']} ({df_dissent.iloc[-1]['avg_dissent_rate']:.1f}%)")

print("\n" + "="*80)
print(f"✅ All visualizations saved to: {GRAPHICS_DIR}")
print("="*80)